In [3]:
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import tqdm
import gspread

scope = ["https://spreadsheets.google.com/feeds",
         "https://www.googleapis.com/auth/spreadsheets",
         "https://www.googleapis.com/auth/drive.file",
         "https://www.googleapis.com/auth/drive"]

creds = ServiceAccountCredentials.from_json_keyfile_name("hah-project-8b41c84e8c4a.json", scope)


spreadsheet_name = "부자재 입출고 재고 시트"
client = gspread.authorize(creds)
spreadsheet = client.open(spreadsheet_name)

## by name
sheet = spreadsheet.worksheet("출고")

print(sheet)

#print(sheet.get_all_values())
#출고 sheet 처리 작업
#cell_data = sheet.acell('B3').value
#print(cell_data)
column_data = sheet.col_values(3)
#print(len(column_data[3:-1])) #지금까지 부자재 출고된 수를 구함. 지금은 4부터 115

maxRow = 3 + len(column_data[3:-1])
tempDate = "temp날짜" #날짜 저장용 변수
real_out_dict = {} #실제 출고 Key : rowIndex, value : 실제 출고 수량
request_out_dict = {} #요청 출고  Key : rowIndex, value : 요청 출고 수량

# 범위(셀 위치 리스트) 가져오기
real_out_list = sheet.range('F'+str(4)+':F'+str(maxRow))
request_out_list = sheet.range('H'+str(4)+':H'+str(maxRow))
#print(real_out_list)

# 범위에서 각 셀 값 가져오기
for cell in real_out_list:
    print(cell.value)

'''
diffList = ['','','출고할때 부족했던 수량']

for i in range(maxRow-3) :
    if real_out_list[i].value == '' : real_out_list[i].value = 0
    if request_out_list[i].value == '' : request_out_list[i].value = 0
        
    diffList.append(int(request_out_list[i].value) - int(real_out_list[i].value))
for i in range(82, maxRow+1):
    sheet.update_acell('G'+str(i+1), diffList[i])
#sheet.update('G4:G115', diffList)
'''
'''
for rowIndex in range(4, maxRow+1, 1):
    real_out_count = sheet.acell('F'+str(rowIndex)).value
    request_out_count = sheet.acell('H'+str(rowIndex)).value
    if real_out_count == '' : real_out_count = 0
    if request_out_count == '' : real_out_count = 0
    real_out_dict[rowIndex] = int(real_out_count)
    request_out_dict[rowIndex] = int(request_out_count)
    print(rowIndex)

for rowIndex in range(4, maxRow+1, 1):
    print(rowIndex)
    sheet.update_acell('G'+str(rowIndex), request_out_dict[rowIndex]-real_out_dict[rowIndex])
    
'''

<Worksheet '출고' id:0>
852
253
1
2
288
317
1
1
1
60
240
60
0
72
300
600
1
1
0
0
600
1200
1
2
110
50
40
600
600
131
200
300
300
0
1
200
200
200
672
300
500
300
300
69
528
300
500
1000
1
300
250
600
1200
1
1
1200
1200
700
1000
600
300
500
600
600
500
600
2
700
1000
600
600
0
1
0
300
600
100
400
250
1
100
1
1
600
600
300
600
1
1
2
120
1000

1200
1200
620
1250
1
2
600
600
300
600
1
1
500
1000
200
200
1
270
540
3
100
400
630
1000
1000
2
1
1
1
1
600
600
300
600
1
500
1000
1200
1200
600
1200
2
2
600
600
300
600
1
500

1
1200
1200
600
1200
1
1
700


"\nfor rowIndex in range(4, maxRow+1, 1):\n    real_out_count = sheet.acell('F'+str(rowIndex)).value\n    request_out_count = sheet.acell('H'+str(rowIndex)).value\n    if real_out_count == '' : real_out_count = 0\n    if request_out_count == '' : real_out_count = 0\n    real_out_dict[rowIndex] = int(real_out_count)\n    request_out_dict[rowIndex] = int(request_out_count)\n    print(rowIndex)\n\nfor rowIndex in range(4, maxRow+1, 1):\n    print(rowIndex)\n    sheet.update_acell('G'+str(rowIndex), request_out_dict[rowIndex]-real_out_dict[rowIndex])\n"

In [35]:
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import tqdm
import gspread
import datetime

scope = ["https://spreadsheets.google.com/feeds",
         "https://www.googleapis.com/auth/spreadsheets",
         "https://www.googleapis.com/auth/drive.file",
         "https://www.googleapis.com/auth/drive"]

creds = ServiceAccountCredentials.from_json_keyfile_name("hah-project-8b41c84e8c4a.json", scope)

spreadsheet_name = "부자재 입출고 재고 시트"
client = gspread.authorize(creds)
#spreadsheet = client.open(spreadsheet_name)

# 접속 가능한 시트 파일 조회  ->  시트별 공유 설정된 파일 조회 가능함
#print(client.list_spreadsheet_files())

sheet_file = client.open('부자재 입출고 재고 시트')
worksheet_list = sheet_file.worksheets()

#현재 시간
#currentTime = datetime.datetime.now()
#DATE = str(currentTime.date()).split('-')
#TIME = str(currentTime.time().replace(microsecond=0)).split(':')
#print(DATE)
#print(TIME)

#실제 출고 dictionary key : 날짜,  value : list[공장명, 부자재명, 출고 수량]
out_dict = {}

#실제 입고 dictionary key : 날짜, value : list[공장명, 부자재명, 입고 수량]
in_dict = {}

df_worksheet_list = []

def makeList(list):
    return [list]

for index in range(0, len(worksheet_list)-1):
    worksheet = sheet_file.get_worksheet(index)   # 시트 인덱스 번호로 지정할때
    #print(worksheet)
    column_data = worksheet.col_values(3)
    maxRow = 4 + len(column_data[3:-1])
    df = pd.DataFrame(worksheet.get_all_values())
    df_worksheet_list.append(df)

    if index == 0 : #출고 sheet
        continue
        requestOut_list = df[3:][7].values.tolist()
        realOut_list = df[3:][5].values.tolist()
        lackList = []
        colorIndexList = []
        #print(realOut_list)
        for i in range(len(realOut_list)) :
            if requestOut_list[i] == "" :
                requestOut_list[i] = 0
            if realOut_list[i] == "" :
                realOut_list[i] = 0
            diff = int(requestOut_list[i]) - int(realOut_list[i])
            lackList.append([diff])
            if diff != 0 :
                colorIndexList.append(i + 4)
        #print(lackList)
        worksheet.update('G4:G'+ str(maxRow), lackList)
        
        for index in colorIndexList :   
            worksheet.format("G" + str(index), {
                "backgroundColor": {
                  "red": 1.0,
                  "green": 1.0,
                  "blue": 0.0
                },
                "textFormat": {
                  "bold": True
                }
            })
    elif index == 1 : #입고 sheet
        continue
    elif index == 2 : #재고 sheet
        continue
    elif index == 3 : #공장별 총 출고 수량 sheet
        maxRow = len(tempList) + 3
        
        #출고날짜 입력
        tempList = df_worksheet_list[0][2:][1].values.tolist()
        tempList = list(map(makeList, tempList))
        worksheet.update('B3:B'+ str(maxRow), tempList)
        
        #공장명 입력
        tempList = df_worksheet_list[0][2:][2].values.tolist()
        tempList = list(map(makeList, tempList))
        worksheet.update('C3:C'+ str(maxRow), tempList)
        
        #부자재명 입력
        tempList = df_worksheet_list[0][2:][4].values.tolist()
        tempList = list(map(makeList, tempList))
        worksheet.update('D3:D'+ str(maxRow), tempList)
        
        #실제 출고수량 입력
        tempList = df_worksheet_list[0][2:][5].values.tolist()
        tempList = list(map(makeList, tempList))
        worksheet.update('E3:E'+ str(maxRow), tempList)
        
    elif index == 4 : #공장별 월별 출고 수량 sheet
        continue
    #df.to_excel("./" + str(worksheet).split('\'')[1] + "(" + DATE[1] + DATE[2] + "_" + TIME[0] + "시" + TIME[1]+"분).xlsx", index = False)
    #df.to_excel("./" + str(worksheet).split('\'')[1]+".xlsx", index = False)

<h1> 출고 시트

In [18]:
#print(df_worksheet_list[0])
#print(df_worksheet_list[1])

print(df_worksheet_list[0][2:][1])
print(df_worksheet_list[0][2:][2])
print(df_worksheet_list[0][2:][4])
print(df_worksheet_list[0][2:][5])

df_worksheet_list[0][2:][1].values.tolist()

2            출고날짜
3      2022.03.08
4                
5                
6                
          ...    
167              
168              
169              
170              
171              
Name: 1, Length: 170, dtype: object
2      공장명
3      수락2
4      수락2
5      수락2
6      수락2
      ... 
167    수락2
168    수락2
169    수락2
170    수락2
171    수락2
Name: 2, Length: 170, dtype: object
2             부자재명
3          정방향슬라이더
4          역방향슬라이더
5        개이빨지퍼(VS)
6               갱지
          ...     
167        정방향슬라이더
168       방수지퍼(RC)
169      개이빨지퍼(VS)
170       hah 메인라벨
171    hah 케어라벨(2)
Name: 4, Length: 170, dtype: object
2      실제 출고수량
3          852
4          253
5            1
6            2
        ...   
167        625
168          1
169          1
170        500
171       1000
Name: 5, Length: 170, dtype: object


['출고날짜',
 '2022.03.08',
 '',
 '',
 '',
 '2022.03.11',
 '',
 '',
 '2022.03.11',
 '',
 '',
 '',
 '',
 '2022.03.14',
 '',
 '',
 '',
 '',
 '',
 '2022.03.15',
 '',
 '',
 '',
 '',
 '',
 '',
 '2022.03.17',
 '',
 '',
 '',
 '2022.03.21',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '2022.03.21',
 '',
 '',
 '2022.03.22',
 '',
 '',
 '',
 '',
 '',
 '',
 '2022.03.28',
 '2022.03.28',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '2022.03.30',
 '',
 '',
 '',
 '',
 '2022.04.05',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '2022.04.08\n ',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '2022.04.11',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '2022.04.12',
 '',
 '2022.04.14',
 '2022.04.14',
 '2022.04.15',
 '',
 '',
 '',
 '',
 '',
 '',
 '2022.04.18',
 '',
 ' 2022.04.19',
 '',
 '',
 '',
 '',
 '',
 '',
 '2022.04.19',
 '',
 '',
 '',
 '',
 '',
 '2022.04.20',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '2022.04.25',
 '',
 '',
 '',
 '',
 '